In [1]:
!pip3 install datasets huggingface-hub

In [2]:
from datasets import load_dataset

dataset = load_dataset("agentsea/wave-ui-25k", num_proc=16)

#does not filter out mobile images right
filtered_dataset = dataset['train'].filter(
    lambda x: x['language'] is not None 
    and 'English' in x['language']
    and x['platform'] is not None 
    and 'web' in x['platform']
    , num_proc = 16
)

print(filtered_dataset.column_names)


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

['image', 'instruction', 'bbox', 'resolution', 'source', 'platform', 'name', 'description', 'type', 'OCR', 'language', 'purpose', 'expectation']


In [3]:
def format_instruction(example):
        instruction = f"""Return the bounding box of the {example['description']}. It's used to {example['purpose']} and if we click it {example['expectation']}."""
        bbox = example['bbox']
        x_res, y_res = example['resolution']
        #bounding boxes are in percentage of the screen in format [x1,y1,x2,y2] 
        bbox[0] = bbox[0] / x_res * 100
        bbox[1] = bbox[1] / y_res * 100
        bbox[2] = bbox[2] / x_res * 100
        bbox[3] = bbox[3] / y_res * 100
        bbox = [round(x, 2) for x in bbox]  # Limit to two decimal places
        #print(bbox)
        reply = f"""{str(bbox)}"""
        
        # Create messages format
        messages = [ { "user": instruction, "assistant": reply, "source": "web_scraper" } ]

        
        return messages


In [4]:
print(filtered_dataset.column_names)

filtered_dataset_formatted = filtered_dataset.map(lambda x: {"texts": format_instruction(x)})
# Drop the 'source' column since it's not needed
filtered_dataset_formatted = filtered_dataset_formatted.remove_columns([ 'instruction', 'bbox', 'resolution', 'source', 'platform', 'name', 'description', 'type', 'OCR', 'language', 'purpose', 'expectation'])
# Rename 'prompt' column to 'texts' to match expected format
filtered_dataset_formatted = filtered_dataset_formatted.rename_column('image', 'images')
print(filtered_dataset_formatted.column_names)




['image', 'instruction', 'bbox', 'resolution', 'source', 'platform', 'name', 'description', 'type', 'OCR', 'language', 'purpose', 'expectation']


Map:   0%|          | 0/15937 [00:00<?, ? examples/s]

['images', 'texts']


In [5]:
print(filtered_dataset_formatted[0]['images'])



<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1280x720 at 0x7F1DD9EE1CD0>


In [7]:
from huggingface_hub import HfApi
# Set up HuggingFace API and push
username = "jwaters8978"
repo_name = "web_scraper_dataset_2"

api = HfApi()
api.create_repo(repo_id=f"{username}/{repo_name}", repo_type="dataset", exist_ok=True)
filtered_dataset_formatted.push_to_hub(f"{username}/{repo_name}")

# Print info
print(f"\nPushed to {repo_name}:")
print(f"Number of rows: {len(filtered_dataset)}")
print("\nSample of filtered data:")
print(filtered_dataset.shuffle(seed=42).select(range(5)))

Uploading the dataset shards:   0%|          | 0/13 [00:00<?, ?it/s]

Map:   0%|          | 0/1226 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Map:   0%|          | 0/1226 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/1226 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Map:   0%|          | 0/1225 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]


Pushed to web_scraper_dataset_2:
Number of rows: 15937

Sample of filtered data:
Dataset({
    features: ['image', 'instruction', 'bbox', 'resolution', 'source', 'platform', 'name', 'description', 'type', 'OCR', 'language', 'purpose', 'expectation'],
    num_rows: 5
})
